In [1]:
import numpy as np
import pandas as pd
import json
import sys
import os
import matplotlib
#matplotlib.use('Agg') 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import seaborn as sns
import pdb
#import ipdb
import h5py
import importlib
import pickle
import math
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import axes3d    

# Add root folder to sys path
sys.path.append("../")

from load_expert_traj import recursively_save_dict_contents_to_group

# %pylab inline
# inline doesn't give interactive plots
# %matplotlib inline 
%matplotlib notebook
plt.rcParams['figure.figsize'] = (8.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'Blues'

sns.set()

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
#%load_ext autoreload
#%autoreload 2

# Import own modules
# Need to use importlib since module starts with a numerical value.

ImportError: No module named 'sklearn'

In [ ]:
def plot_pca_hopper(pkl_path):
    with open(pkl_path, 'rb') as pkl_f:
        results_data = pickle.load(pkl_f)
    save_data_path = '../mujoco_results/'
    
    state_list, context_list = [], []

    for i in range(len(results_data['pred_traj_state'])):
        pred_traj = results_data['pred_traj_state'][i]
        traj_context = np.argmax(results_data['pred_context'][i].squeeze(), axis=1)
    
        state_list.append(pred_traj)
        context_list.append(traj_context)
    
        true_traj = np.array(results_data['true_traj_state'][0][i])[:,0,:]
        true_traj = np.reshape(true_traj, (true_traj.shape[0], -1))

        colors = cm.rainbow(np.linspace(0, 1, pred_traj.shape[0]))
        sns.palplot(colors)
    
        fig = plt.figure(figsize=(8, 8))
        ax = fig.add_subplot(1, 1, 1)
        ax.scatter(pred_traj[:, 0].tolist(), pred_traj[:, 1].tolist(),
                 color=colors, s=16) 
        ax.axis('equal')    
        ax.axes.xaxis.set_ticklabels([])
        ax.axes.yaxis.set_ticklabels([])
        ax.set_title("Predicted Trajectory", fontsize=18)

        plt.savefig(
            '{}/result_{}.pdf'.format(save_data_path, i),
            bbox_inches='tight',
        )
        plt.show()
          
        plt.scatter(true_traj[:, 0].tolist(), true_traj[:, 1].tolist(), color=colors, s=8) 
        plt.axis('equal')    
        plt.title("True traj")
        plt.show()                    

        pred_context = np.argmax(
            np.reshape(np.array(results_data['pred_context'][0][i]), (119, -1)), axis=1)
        plt.plot(pred_context.tolist())
        plt.show()    

def plot_pca_walker(pkl_path):
    with open(pkl_path, 'rb') as pkl_f:
    results_data = pickle.load(pkl_f)
    save_data_path = '../mujoco_results/'
    
    state_list, context_list = [], []

    for i in range(len(results_data['pred_traj_state'])):
        pred_traj = results_data['pred_traj_state'][i]
        traj_context = np.argmax(results_data['pred_context'][i].squeeze(), axis=1)
    
        state_list.append(pred_traj)
        context_list.append(traj_context)
    
    state_array = np.array(state_list)
    state_array = np.reshape(state_array, 
                                 (state_array.shape[0]*state_array.shape[1], state_array.shape[2]))
    context_array = np.array(context_list)
    context_array = np.reshape(context_array, 
                           (context_array.shape[0]*context_array.shape[1],))

        context_colors = ['g', 'b', 'r', 'm', 'y', 'c', 'k', 'w']
        traj_context_color = [context_colors[j] for j in context_array[:]]

        pca = PCA(n_components=3)
        pca.fit(state_array)

    print(pca.explained_variance_ratio_)
    state_transformed = pca.fit_transform(state_array)

    context_array = context_array.astype(np.int32)

    fig = plt.figure(figsize=(8.0,8.0))
    ax = fig.add_subplot(111, projection='3d')

    ax.scatter(state_transformed[:, 0][context_array == 0],
               state_transformed[:, 1][context_array == 0],
               state_transformed[:, 2][context_array == 0],
               color='g',
               label='Context 1',
               s=8)
    ax.scatter(state_transformed[:, 0][context_array == 1],
               state_transformed[:, 1][context_array == 1],
               state_transformed[:, 2][context_array == 1],
               color='b',
               label='Context 2',
               s=8)
    ax.scatter(state_transformed[:, 0][context_array == 2],
               state_transformed[:, 1][context_array == 2],
               state_transformed[:, 2][context_array == 2],
               color='r',
               label='Context 3',
               s=8)


    ax.axis('equal')   
    ax.axes.xaxis.set_ticklabels([])
    ax.axes.yaxis.set_ticklabels([])
    ax.axes.zaxis.set_ticklabels([])
    ax.set_title("PCA")
    ax.legend(prop={'size': 20})

    if save_path is not None:
        plt.savefig(save_path, bbox_inches='tight')
    plt.show()
    

#pkl_path = '../results/hopper/discrete_vae/batch_64_context_4_no_time/results/results_cp_640.pkl'
# save_path = '../mujoco_results/pca_hopper_expert.pdf'

pkl_path = '../results/walker_2d/discrete_vae/expert_trajs_100/batch_64_context_3_no_time/results/results_cp_1000.pkl'
save_path = '../mujoco_results/pca_walker_expert.pdf'

plot_pca_hopper('../results/hopper/discrete_vae/batch_64_context_4_no_time/results/results_cp_640_expert_states.pkl')
